# Main Notebook

File to perform experiments

## Imports

In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

In [ ]:
import os
import numpy as np
import gpflow
import matplotlib.pyplot as plt
import pandas as pd
import time
from tqdm import tqdm

from models.GaussianProcess import GaussianProcess
from acquisition_functions.MES import mes_acq, basic_mes_acq
from acquisition_functions.PESMO import pesmo_acq
from acquisition_functions.MESMO import mesmo_acq
from arguments.arguments import MainArguments

from MOObenchmark import MOOackley, MOOcrossit, MOOquadratic
from utils.calc_pareto import get_pareto_undominated_by, getSetfromFront

from models.MOOEvaluationProblem import MOOEvaluationProblem

from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.factory import get_termination
from pymoo.optimize import minimize

## Algorithm Arguments

In [ ]:
eval_function = "MOOackley/"
acq_function  = "basic_mes_acq"
exp_id = 89
d = 1
    
seed = 0
np.random.seed(seed)

lower_bound = -2
upper_bound = 2

lowerBounds = [lower_bound]*d
upperBounds = [upper_bound]*d


## Evaluation

In [ ]:
T=1.2

def evaluation(x):
    return MOOackley(x, c1=-T/2, c2=T/2)

N = 1_001
X = np.linspace(lower_bound,upper_bound,N)
Z = np.zeros((N,2))

O = 2
C = 0

problem = MOOEvaluationProblem(evaluation, lowerBound=lower_bound, upperBound=upper_bound)
algorithm = NSGA2()
res = minimize( problem, 
                algorithm,
                termination = get_termination("n_gen",100))

real_pareto = res.F[np.argsort(res.F[:,0])]

for i in range(N):
    Z[i]=evaluation(X[i])

fig, axs = plt.subplots(1,3, figsize=(15,4))

axs[0].plot(X, Z[:,0], 'b')
axs[0].plot(X, Z[:,1], 'k')
axs[0].plot(res.X, res.F[:,0], 'xr', markersize=5)
axs[0].plot(res.X, res.F[:,1], 'xr', markersize=5)

axs[1].plot(np.reshape(Z,(-1,2))[:,0], np.reshape(Z,(-1,2))[:,1], 'kx')
axs[1].plot(res.F[:,0], res.F[:,1], 'rx')

axs[2].plot(res.F[:,0], res.F[:,1], 'x')
plt.show()

In [ ]:
def random_acq(GP: GaussianProcess, **kwargs):
    while True:
        x_rand = np.random.uniform(GP.lowerBounds, GP.upperBounds, GP.d)
        if GP.X is None or not x_rand in GP.X:
            break
    return x_rand, 0

## Experiments

### Read Previous experiment

In [ ]:
df = pd.read_csv(eval_function+acq_function+".csv")
df = df[df['exp_id']==exp_id]

In [ ]:
def string_to_numpy(str):
    str=str[1:-1]
    values = []
    for v in str.split(" "):
        if v=="":
            continue
        values.append(float(v))
    return np.array(values)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(df['ns'], df['hp_e']-df['hp_r'], label="hp")
plt.plot(df['ns'], df['d_e_r'], label="d")
plt.legend()

In [ ]:
df_metrics = None
with tf.device('/GPU:1'):
    ## Kernerl configuration 
    k = gpflow.kernels.SquaredExponential()
    ### GPs Initialization
    GP = GaussianProcess(O, C, d, lowerBounds, upperBounds, k, noise_variance=2e-6)

    for n, i in enumerate(df.itertuples()):

        x = string_to_numpy(i.x)
        y = string_to_numpy(i.y)
        print(n, ":  ", x, y)
        
        ## UPDATE
        GP.addSample(x, y)     ## Add new sample to the model
        GP.updateGP()                  ## Update data on the GP regressor
        GP.optimizeKernel()             ## Optimize kernel hyperparameters
        GP.plotSamples()
        
        ## Evaluate Pareto (distances and hypervolumes)
        metrics = GP.evaluatePareto(real_pareto, showparetos = True, saveparetos = False)
        metrics['ns'] = len(GP.X)
        if df_metrics is None:
            df_metrics = pd.DataFrame({k: [v] for k, v in metrics.items()})
        else:
            df_metrics = pd.concat([df_metrics, pd.DataFrame({k: [v] for k, v in metrics.items()})])

In [ ]:
plt.plot(df_metrics['ns'], abs(df_metrics['hp_e']-df_metrics['hp_r']), label="hp")
plt.plot(df_metrics['ns'], df_metrics['d_e_r'], label="d")
plt.legend()